# Explorative Datenanalyse (EDA) - Palmer Penguins
In diesem Notebook analysieren wir den Palmer-Penguin-Datensatz, um fundamentale Entscheidungen für das **Software-Design** und die **Modell-Architektur** zu treffen.

**Ziele:**
1. **Datenqualität:** Identifikation von Missing Values und Ausreißern zur Definition von Preprocessing-Pipelines.
2. **Feature-Analyse:** Bewertung der Features hinsichtlich Trennschärfe und Notwendigkeit (User Experience).
3. **Validierung:** Ableitung technischer Grenzen für die Benutzeroberfläche (UI).

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_theme(style="whitegrid")
warnings.filterwarnings('ignore')

PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.penguin_classifier import config
from penguin_classifier.dataset import load_data
import ydata_profiling

2026-01-06 16:03:58.487 | INFO     | penguin_classifier.config:<module>:11 - PROJ_ROOT path is: C:\Users\Erik\PycharmProjects\penguin_classifier
2026-01-06 16:03:58.488 | INFO     | penguin_classifier.config:<module>:16 - data path is: C:\Users\Erik\PycharmProjects\penguin_classifier\data


## 1. Daten laden und erste Inspektion

In [4]:
df = load_data(config.RAW_DATA_PATH)
df.head()

2026-01-06 16:04:00.096 | INFO     | penguin_classifier.dataset:load_data:28 - Data loaded from C:\Users\Erik\PycharmProjects\penguin_classifier\data\raw\data.csv


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


**Beobachtung & Entscheidung:**
Der Datensatz enthält Features zu Schnabelmaßen, Flossenlänge, Körpermasse sowie Geschlecht und Insel.
Das Feature `year` repräsentiert lediglich den Zeitpunkt der Datenerhebung und kein biologisches Merkmal des Tiers. Da die Software zeitunabhängig funktionieren soll, stellt dieses Feature Rauschen dar und birgt die Gefahr, dass das Modell irrelevante zeitliche Korrelationen lernt.
**Konsequenz:** Wir schließen `year` explizit aus dem Training aus, um die Robustheit zu erhöhen und die Eingabemaske der Software zu vereinfachen.

In [ ]:
df.info()

In [10]:
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,342.000000,342.000000,342.000000,342.000000,344.000000
mean,43.921930,17.151170,200.915205,4201.754386,2008.029070
std,5.459584,1.974793,14.061714,801.954536,0.818356
min,32.100000,13.100000,172.000000,2700.000000,2007.000000
25%,39.225000,15.600000,190.000000,3550.000000,2007.000000
50%,44.450000,17.300000,197.000000,4050.000000,2008.000000
75%,48.500000,18.700000,213.000000,4750.000000,2009.000000
max,59.600000,21.500000,231.000000,6300.000000,2009.000000


## 2. Analyse fehlender Werte (Missing Values)

In [ ]:
df.isnull().sum()

In [ ]:
df[df.isnull().any(axis=1)]

**Analyse für das Pipeline-Design**: Das Feature `sex` weist fehlende Werte auf. Während wir im Training Zeilen löschen können, müssen wir für die Software eine Entscheidung treffen: Was passiert, wenn der Nutzer vor Ort das Geschlecht nicht bestimmen kann oder keine Angabe dazu macht?

Option A (Drop): Die App verweigert die Vorhersage -> schlechte User Experience.

Option B (Imputation): Wir trainieren einen Imputer (z. B. Häufigster Wert), der im Hintergrund läuft.

Option C (Kategorie 'Unknown'): Wir behandeln fehlende Werte als eigene Information.

**Design-Entscheidung:**
Um die Software robust und benutzerfreundlich zu halten, definieren wir folgende Strategie:
* **Training:** Zeilen mit fehlenden essenziellen Messwerten (alle numerischen außer `year`) werden entfernt, da diese Features für die Klassifikation zwingend erforderlich sind.
* **Inferenz:** Für das Feature `sex` integrieren wir einen `SimpleImputer (strategy='constant', fill_value='unknown')` in die Pipeline. Dies verhindert Abstürze, falls das Geschlecht vor Ort nicht bestimmt werden kann sondern bietet die Möglichkeit, "unknown" anzugeben.

## 3. Analyse der Zielvariable (Target)
Wie sind die Pinguin-Spezies verteilt? Gibt es Class-Imbalances?

In [ ]:
sns.countplot(df["species"], palette="viridis")

**Beobachtung:**
Die Spezies *Adelie* dominiert, *Chinstrap* ist unterrepräsentiert. Der Datensatz ist leicht unbalanciert.
**Konsequenz:**
Wir verwenden für die Modell-Validierung einen **`StratifiedKFold`**. Dies garantiert, dass alle Klassen in den Trainings- und Test-Folds repräsentativ verteilt sind und verhindert verzerrte Metriken.

## 4. Zusammenhang zwischen Features und Spezies
 Es wird untersucht, welche Features die Spezies am besten trennen.

### 4.1 Geografische Verteilung (Insel-Effekt)

In [ ]:
sns.countplot(x='island', hue='species', data=df)
plt.title("Vorkommen der Spezies nach Insel")
plt.show()

**Erkenntnis:** Die Insel ist ein starker Prädiktor (*Gentoo* nur auf Biscoe, *Chinstrap* nur auf Dream). Da es nur drei statische Kategorien gibt, implementieren wir dieses Feature in der UI als **Dropdown-Menü**. Dies minimiert Fehleingaben (im Vergleich zu Freitext) und maximiert die Datenqualität bei der Eingabe.

### 4.2 Numerische Features vs. Spezies & Geschlecht
Prüfen der Verteilungen der physischen Merkmale.

In [ ]:
features = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes_flat = axes.flatten()

for i, feature in enumerate(features):
    sns.boxplot(
        x="species", 
        y=feature, 
        hue="sex", 
        data=df, 
        ax=axes_flat[i],
    )
    axes_flat[i].set_title(f"Verteilung von {feature}")
plt.tight_layout()
plt.show()

**Beobachtungen & System-Implikationen:**

1.  **Trennschärfe:** Die Boxplots zeigen eine klare Separierbarkeit der Spezies, insbesondere durch `flipper_length_mm` und `bill_length_mm`. Dies bestätigt die Machbarkeit eines robusten Klassifikators mit diesen Features.
2.  **Sexualdimorphismus:** Es ist ein systematischer Unterschied zwischen männlichen und weiblichen Tieren erkennbar (Männchen sind tendenziell schwerer und größer). Das Feature `sex` besitzt somit eine hohe Vorhersagekraft in Kombination mit den Messwerten.
    * *Design-Entscheidung:* `sex` muss zwingend als Eingabefeld in die UI aufgenommen werden.
3.  **Ausreißer (Outliers):** Die Plots zeigen vereinzelte Ausreißer.
    * *Design-Entscheidung:* Für das Preprocessing empfiehlt sich ein `RobustScaler` oder eine Outlier-Bereinigung, um die Modellstabilität zu gewährleisten. Zudem definieren diese Extremwerte die **Validierungsgrenzen (Min/Max)** für die Eingabefelder im Frontend, um Falscheingaben zu verhindern.

**Berechnung von Validierungsgrenzen für das Frontend (UI)**
nutzen min/max mit einem Puffer, um unrealistische Eingaben in der App abzufangen.

In [8]:
# Berechnung von Validierungsgrenzen für die UI (Frontend)
# Wir nutzen min/max mit einem Puffer,
# um unrealistische Eingaben in der App abzufangen.

validation_rules = {}
for col in ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]:
    min_val = df[col].min()
    max_val = df[col].max()
    # Puffer von 10% für die UI-Slider
    buffer = (max_val - min_val) * 0.2
    
    validation_rules[col] = {
        "min_ui": round(min_val - buffer, 1),
        "max_ui": round(max_val + buffer, 1),
        "min_hard": 0.0
    }

print("Vorgeschlagene UI-Ranges für Config/Pydantic:")
pd.DataFrame(validation_rules).T

Vorgeschlagene UI-Ranges für Config/Pydantic:


,min_ui,max_ui,min_hard
bill_length_mm,29.4,62.4,0.0
bill_depth_mm,12.3,22.3,0.0
flipper_length_mm,166.1,236.9,0.0
body_mass_g,2340.0,6660.0,0.0


#### Ableitung von Validierungsregeln (Frontend)
Wir berechnen hier die `min` und `max` Werte zzgl. eines Puffers. Diese Werte dienen als **Hard Constraints** für die Slider/Input-Felder in der UI (z.B. Pydantic Models oder Dash-Config), um unrealistische Eingaben physikalisch unmöglicher Pinguine abzufangen.

### 4.3 Korrelationsanalyse
Prüfung auf Multikollinearität zwischen den numerischen Features.

In [ ]:
df_nums = df.select_dtypes(include=['float64', 'int64']).drop(columns=["year"], errors='ignore')
sns.heatmap(df_nums.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Korrelationsmatrix der numerischen Features")
plt.show()

**Erkenntnis:**
Es liegt eine starke Multikollinearität (> 0.87) zwischen `flipper_length_mm` und `body_mass_g` vor.

**System-Design-Implikation:**
Aus Effizienzgründen könnte eines der Features entfernt werden, um die Eingabemaske zu verschlanken. Wir entscheiden uns jedoch vorerst, **beide Features zu behalten**, da wir keine Rechenleistungsprobleme erwarten. Wir markieren dies jedoch als Option für eine spätere Optimierung (Feature Selection), falls die User Experience durch zu viele Eingabefelder leidet.

Pairplot für den Gesamtüberblick

In [ ]:
sns.pairplot(df[["species", "bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]], hue="species", palette="viridis")

In [ ]:
profile = ydata_profiling.ProfileReport(df, title="Penguin Data Profiling Report", explorative=True)
output_path = config.REPORTS_DIR / "penguin_raw_data_profile.html"
profile.to_file(output_path)

## 5. Fazit & Nächste Schritte

Die EDA liefert folgende Vorgaben für die **Konzeptionsphase**:

1.  **Daten:** Wir nutzen `bill_length`, `bill_depth`, `flipper_length`, `body_mass`, `sex` und `island`.
2.  **Preprocessing:**
    * `SimpleImputer` für `sex`.
    * `RobustScaler` für numerische Werte (wegen Ausreißern).
    * `OneHotEncoder` für `island` und `sex`.
3.  **UI-Constraints:** Die berechneten Min/Max-Werte werden in die Config übernommen.
4.  **Validierung:** Nutzung von `StratifiedKFold`.

**Nächster Schritt:** Erstellung des technischen Konzepts (Pipeline-Architektur & API-Design).